## Setup

Run the commands below and manually install the KD_Lib library; Note that unstalling it directly from pip will fail due to an [open issue](https://github.com/SforAiDl/KD_Lib/issues/108).

In [32]:
!git clone https://github.com/SforAiDl/KD_Lib.git
!cd KD_Lib && python3 setup.py install --user

fatal: destination path 'KD_Lib' already exists and is not an empty directory.
running install
running bdist_egg
running egg_info
writing KD_Lib.egg-info/PKG-INFO
writing dependency_links to KD_Lib.egg-info/dependency_links.txt
writing requirements to KD_Lib.egg-info/requires.txt
writing top-level names to KD_Lib.egg-info/top_level.txt
reading manifest file 'KD_Lib.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'KD_Lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/KD_Lib
copying build/lib/KD_Lib/__init__.py -> build/bdist.linux-x86_64/egg/KD_Lib
creating build/bdist.linux-x86_64/egg/KD_Lib/Pruning
copying build/lib/KD_Lib/Pruning/__init__.py -> build/bdist.linux-x86_64/egg/KD_Lib/Pruning
creating build/bdist.linux-x86_64/egg/KD_Lib/Pruning/lottery_tickets
copying build/lib/KD_Lib/Pruning/lottery_tickets/lot

byte-compiling build/bdist.linux-x86_64/egg/KD_Lib/Pruning/lottery_tickets/__init__.py to __init__.cpython-36.pyc
byte-compiling build/bdist.linux-x86_64/egg/KD_Lib/KD/vision/CSKD/cskd.py to cskd.cpython-36.pyc
byte-compiling build/bdist.linux-x86_64/egg/KD_Lib/KD/vision/CSKD/__init__.py to __init__.cpython-36.pyc
byte-compiling build/bdist.linux-x86_64/egg/KD_Lib/KD/vision/CSKD/sampler.py to sampler.cpython-36.pyc
byte-compiling build/bdist.linux-x86_64/egg/KD_Lib/KD/vision/TAKD/takd.py to takd.cpython-36.pyc
byte-compiling build/bdist.linux-x86_64/egg/KD_Lib/KD/vision/TAKD/__init__.py to __init__.cpython-36.pyc
byte-compiling build/bdist.linux-x86_64/egg/KD_Lib/KD/vision/noisy/__init__.py to __init__.cpython-36.pyc
byte-compiling build/bdist.linux-x86_64/egg/KD_Lib/KD/vision/noisy/soft_random.py to soft_random.cpython-36.pyc
byte-compiling build/bdist.linux-x86_64/egg/KD_Lib/KD/vision/noisy/noisy_teacher.py to noisy_teacher.cpython-36.pyc
byte-compiling build/bdist.linux-x86_64/egg/K

In [1]:
import json
from pprint import pprint

import cifar100_helper  # local

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision

from sklearn.metrics import (classification_report, accuracy_score, confusion_matrix,
                             ConfusionMatrixDisplay)

In [2]:
# Use this to view all the available models
# pprint(torch.hub.list("chenyaofo/pytorch-cifar-models", force_reload=True))
student_model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_resnet20", pretrained=False)
teacher_model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_resnet56", pretrained=True)

Using cache found in /home/bruno/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master
Using cache found in /home/bruno/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


In [3]:
train_loader, test_loader = cifar100_helper.cifar100("./", 32, None, None, batch_size=10000, num_workers=4, is_vit=True)

idx_to_class = {v:k for k, v in train_loader.dataset.class_to_idx.items()}
idx_to_class_vec = np.array(list(idx_to_class.values()))

# functions to show an image
def imshow(img, to_numpy=True):
    img = img / 2 + 0.5     # unnormalize
    if to_numpy:
        npimg = img.numpy()
    else:
        npimg = img
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    return img

# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def print_acc(model, loader):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in loader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = model(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    print(f'Accuracy of the network on the {len(loader.dataset)} test images: %d %%' % (100 * correct / total))

In [7]:
# results for our model
print("teacher model ", end="")
print_acc(teacher_model, test_loader)
# we except this to be ~0% or 1%
print("student model ", end="")
print_acc(student_model, test_loader)

teacher model Accuracy of the network on the 10000 test images: 67 %
student model Accuracy of the network on the 10000 test images: 0 %


## Knowledge Distilation

In [32]:
import torch.optim as optim

# These values are the same from the original training of our teacher model
# https://github.com/chenyaofo/image-classification-codebase/blob/master/conf/cifar10.conf
# Note that although the file is for CIFAR-10, the CIFAR-100 file inherits theses parameters.
optim_dict = dict(
    lr=0.1,
    momentum=0.9,
    dampening=0,
    weight_decay=5e-4,
    nesterov=True,
)


student_optimizer = optim.SGD(student_model.parameters(), **optim_dict)
# we don't use this optimizer since we end up training just the student,
# but we must pass in a valid value
teacher_optimizer = optim.SGD(student_model.parameters(), **optim_dict)

In [33]:
%%time
from KD_Lib.KD import VanillaKD
# Now, this is where KD_Lib comes into the picture
distiller = VanillaKD(
    teacher_model, student_model, train_loader, test_loader, teacher_optimizer, student_optimizer,
    temp=20.0, distil_weight=0.5, device="cpu", log=False, logdir="./Experiments",
)
distiller.train_student(epochs=1, plot_losses=True, save_model=False)    # Train the student network
distiller.evaluate(teacher=False)                                       # Evaluate the student network
# distiller.get_parameters()                  

Training Student...


KeyboardInterrupt: 

In [6]:
# get some random test images
dataiter = iter(test_loader)
images, labels = dataiter.next()

outputs = teacher_model(images)
_, predicted = torch.max(outputs, 1)

## Log results/metrics for this single batch 

Write metrics

In [7]:
results = classification_report(labels, predicted, output_dict=True)
# results = {idx_to_class[int(k)]: v for k,v in results.items() if k.isnumeric()}
# 
# with open('metrics.json', 'w') as fp:
#     json.dump(results, fp, sort_keys=True, indent=4)
#     
# with open('accuracy.json', 'w') as fp:
#     json.dump({"accuracy": accuracy_score(labels, predicted)}, fp)

In [8]:
results

{'0': {'precision': 0.8303571428571429,
  'recall': 0.93,
  'f1-score': 0.8773584905660378,
  'support': 100},
 '1': {'precision': 0.8837209302325582,
  'recall': 0.76,
  'f1-score': 0.8172043010752689,
  'support': 100},
 '2': {'precision': 0.5533980582524272,
  'recall': 0.57,
  'f1-score': 0.5615763546798028,
  'support': 100},
 '3': {'precision': 0.42857142857142855,
  'recall': 0.48,
  'f1-score': 0.4528301886792452,
  'support': 100},
 '4': {'precision': 0.4824561403508772,
  'recall': 0.55,
  'f1-score': 0.5140186915887851,
  'support': 100},
 '5': {'precision': 0.72, 'recall': 0.72, 'f1-score': 0.72, 'support': 100},
 '6': {'precision': 0.7142857142857143,
  'recall': 0.75,
  'f1-score': 0.7317073170731706,
  'support': 100},
 '7': {'precision': 0.6632653061224489,
  'recall': 0.65,
  'f1-score': 0.6565656565656566,
  'support': 100},
 '8': {'precision': 0.9,
  'recall': 0.81,
  'f1-score': 0.8526315789473685,
  'support': 100},
 '9': {'precision': 0.626984126984127,
  'recall'

Write correlations

In [11]:
cm = pd.DataFrame(confusion_matrix(labels, predicted), columns=idx_to_class.values(), index=idx_to_class.values())
cm.to_json("confusion_matrix_100.json", indent=True)

Now for the 20 classes version correlation:

In [12]:
# from https://github.com/ryanchankh/cifar100coarse
def sparse2coarse(targets):
    """Convert Pytorch CIFAR100 sparse targets to coarse targets.
    Usage:
        trainset = torchvision.datasets.CIFAR100(path)
        trainset.targets = sparse2coarse(trainset.targets)
    """
    coarse_labels = np.array([ 4,  1, 14,  8,  0,  6,  7,  7, 18,  3,  
                               3, 14,  9, 18,  7, 11,  3,  9,  7, 11,
                               6, 11,  5, 10,  7,  6, 13, 15,  3, 15,  
                               0, 11,  1, 10, 12, 14, 16,  9, 11,  5, 
                               5, 19,  8,  8, 15, 13, 14, 17, 18, 10, 
                               16, 4, 17,  4,  2,  0, 17,  4, 18, 17, 
                               10, 3,  2, 12, 12, 16, 12,  1,  9, 19,  
                               2, 10,  0,  1, 16, 12,  9, 13, 15, 13, 
                              16, 19,  2,  4,  6, 19,  5,  5,  8, 19, 
                              18,  1,  2, 15,  6,  0, 17,  8, 14, 13])
    return coarse_labels[targets]

In [13]:
idx_to_class20 = {i:x for i,x in enumerate([
    'aquatic mammals', 'fish', 'flowers', 'food containers', 'fruit and vegetables', 'household electrical devices', 'household furniture', 'insects', 
     'large carnivores', 'large man-made outdoor things', 'large natural outdoor scenes', 'large omnivores and herbivores', 'medium-sized mammals', 'non-insect invertebrates', 
    'people', 'reptiles', 'small mammals', 'trees', 'vehicles 1', 'vehicles 2'
])}

In [14]:
cm = pd.DataFrame(confusion_matrix(sparse2coarse(labels), sparse2coarse(predicted)), columns=idx_to_class20.values(), index=idx_to_class20.values())
cm.to_json("confusion_matrix_20.json", indent=True)

## Generate feature maps

The visualization code from [utkuozbulak/pytorch-cnn-visualizations](https://github.com/utkuozbulak/pytorch-cnn-visualizations#gradient-visualization) did not work to the model not being currently supported; see [this](https://github.com/utkuozbulak/pytorch-cnn-visualizations/issues/82) issue

In [15]:
# Import M3d-CAM
from medcam import medcam
from torchvision.utils import save_image

In [16]:
%time
amount = 100

!rm attention_maps/ -r
!rm images/ -r
!mkdir images


# We must use batch_size = 1 for this to work
temp_loader = cifar100_helper.cifar100("./", 32, None, None, batch_size=1, num_workers=1, is_vit=True, shuffle=False)[1]
model = medcam.inject(teacher_model, output_dir="attention_maps", save_maps=True, backend="gbp", )

model.eval()
for idx, (batch_x, batch_y) in enumerate(temp_loader):
    if idx == amount:
        break
    # Every time forward is called, attention maps will be generated and saved in the directory "attention_maps"
    save_image(batch_x[0].detach(), f"images/{idx}.png")
    output = model(batch_x)

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 11.4 µs
Files already downloaded and verified
Files already downloaded and verified


/home/bruno/.local/lib/python3.6/site-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
